In [96]:
from openai import OpenAI
import os
import base64
import requests

from dotenv import load_dotenv
# Load API key
_ = load_dotenv()

In [97]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [98]:
client = OpenAI()

### 1. Text generation

* Calling ChatGPT using the API

In [99]:
# to get a response must end on a user reponse
messages = [
    {"role": "system", 
     "content": '''
                   You are a poetic assistant, skilled in explaining complex programming concepts 
                   with creative flair.
                '''},
    
    {"role": "user", 
     "content": '''
                 Compose a poem that explains why the sky is blue in eight lines.
                '''}
     
  ]

In [100]:

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages
)

response = completion.choices[0].message.content

In [101]:
print(response)

In daylight's dance, the sky dons hues of blue,  
As sunlight weaves through air, a magic true.  
Rayleigh's scattering acts as light's gentle sieve,  
Short waves of blue, more wily, twist and live.  

The azure blooms as longer waves drift away,  
Leaving the sky in a cerulean display.  
Thus, in daylight's tapestry, blue shines with grace,  
A secret whispered by the sun's embrace.  


## Image-to-text

<img src="test_image.jpg" width=400/>


In [102]:
img_path = "/Commjhub/jupyterhub/home/angelicahu10/comm4190_F25/comm4190_F25_Using_LLMs_Blog/posts/003_professor_image/test_image.jpg"
base64_image = encode_image(img_path)
prompt = 'Describe this image'
max_tokens = 100

* As part of the user message you can include the image along with a text prompt to direct how the model should analyze it.

In [111]:
models = client.models.list()

for model in models.data:
    print(model.id)

gpt-4-0613
gpt-4
gpt-3.5-turbo
gpt-audio
gpt-5-nano
gpt-audio-2025-08-28
gpt-realtime
gpt-realtime-2025-08-28
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
o1-mini-2024-09-12
o1-mini
gpt-4o-realtime-preview-2024-10-01
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
computer-use-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
computer-use-previe

In [112]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
          "role": "user",
          "content": [
            {"type": "text", "text": prompt},
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              },
            },
          ],
        }
      ],
      max_tokens=max_tokens,
    )

BadRequestError: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}

* The response object contains a lot of information returned by the API call

In [104]:
response

ChatCompletion(id='chatcmpl-CDwAqoQDkta7HqdsyDO6o36984aYB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The image shows a person wearing a neck brace and looking at the camera. They have some visible injuries on their face, including cuts and bruises. Their right hand is bandaged, and they are holding it up with three fingers extended. The background appears to be indoors with a door and some light coming through.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1757437920, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_cbf1785567', usage=CompletionUsage(completion_tokens=63, prompt_tokens=775, total_tokens=838, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cach

* You can pick out just the message

In [105]:
print(response.choices[0].message.content)

The image shows a person wearing a neck brace and looking at the camera. They have some visible injuries on their face, including cuts and bruises. Their right hand is bandaged, and they are holding it up with three fingers extended. The background appears to be indoors with a door and some light coming through.


## Text to image

* Various models will also generate an image based on a text description passed in the API call

In [106]:
image_prompt1 = '''
            create an image of a man holding his hand up'''




In [107]:
response = client.images.generate(
  model="dall-e-3",
  prompt=image_prompt1,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

* The generated image will be temporary available on a cloud server

In [109]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-OGTziyXQKv8dkYauRD1K0Osu/user-jPwUdbyoM52WOxjQGrtmBHI6/img-PIHE3jWQagNqeNkB0RO91xNW.png?st=2025-09-09T16%3A12%3A46Z&se=2025-09-09T18%3A12%3A46Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=52f8f7b3-ca8d-4b21-9807-8b9df114d84c&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-09-09T17%3A12%3A46Z&ske=2025-09-10T17%3A12%3A46Z&sks=b&skv=2024-08-04&sig=D8wVb5cXDk9%2Bptx0fCRrDbMj3B/aJNGZnRvN9zdeohs%3D'

* But most often you will want to download it as part of your script

In [110]:
generated_image = requests.get(image_url)

with open('generated_img.png', 'wb') as out:
    out.write(generated_image.content)